In [1]:
import env
from google.cloud import storage
import json
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models

In [2]:
client = storage.Client()
bucket = client.bucket(env.bucket_patient_hist)
blob = bucket.list_blobs()

In [3]:
print(blob)

In [4]:
blob_list=[]

In [5]:
for each in blob:
    blob_list.append(each.name)

In [6]:
file_data=[]

In [7]:
for file in blob_list:
    blob = bucket.get_blob(file)
    # load blob using json
    file_data.append(json.loads(blob.download_as_string()))

In [8]:
file_data

[{'name': 'Sara',
  'age': 10,
  'language': 'Dutch',
  'sex': 'female',
  'interests': ['pink teddy bears', 'horses', 'playing with ballons'],
  'symptomps': ['bone pain',
   'limping',
   'sometimes she refuses to walk',
   'bruising frequently',
   'small red spots in the skin',
   'fevers that last for days with no evident cause',
   'infections in her mouth'],
  'diagnosis': 'Possible leukemia',
  'treatment': "We need to do a bone marrow biopsy, the procedure consist on doing a small bone marrow puncture to extract cells to understand that blood production is going on as expected, if it isn't, it can be stablish what kind of malfunction is happening by examining the quality of the extracted cells. The procedure implies giving some anesthesia to the child, making her sleep, puncture her space in between a couple of vertebra, extract the bone marrow liquid and then finish. It is a short procedure but it is very important to confirm leukemia",
  'sideeffects': 'A bit of back scratch

In [9]:
def generate(text1,generation_config,safety_settings):
    vertexai.init(project="qwiklabs-gcp-01-e6a034235718", location="us-central1")
    model = GenerativeModel("gemini-1.5-pro-preview-0409")
    responses = model.generate_content(
      [text1],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=True,
    )
    story=''
    for response in responses:
        print(response.text, end="")
        story+=' ' +response.text
    return story


In [10]:
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}

In [11]:
safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}


In [22]:
prompt_info=file_data[1]

In [15]:
prompt_info['language']

'Dutch'

In [23]:
text1 = f"""
#ROLE

You have been transformed to a children's book writer. You work in a children's hospital, and you are specialized in breaking down complex health diagnoses information into stories that can be understood by children. 

#CONTEXT

When a child is diagnosed with a serious decease, it's often hard  for parents to explain that information to the child in an understanding manner. A first diagnosis often contains a lot of technical information and terms, difficult to explain side effect and multiple scenario's for treatment that need to be broken down into a story that a child can understand. 

#INSTUCTIONS

- It is your task to take the all the input we have on a patient, like diagnosis, treatment plan and possible side effects ,and translate into a children's story in {prompt_info['language']}.
- your goal is to create a story which as positive as possible and is tailored to the understanding and interest of the child but explains the situation in full. 
- Make sure that you use an analogy to explain this situation, do not use any technical terms
- Make sure you use a minimum of 1000 words
- Let's think step by step to make sure this story in coherent, correct and useful 

#PATIENT INPUT
Take the following available information on the patient to create your children's story

**Name of the patient**: {prompt_info['name']}

**Age of the patient**: {prompt_info['age']}

**Gender of the patient:** {prompt_info['sex']}

**Interests of the patient**: {prompt_info['interests']}

**Diagnoses of the patient:** {prompt_info['diagnosis']}

**Possible symptoms of decease:** {prompt_info['symptomps']}

**Treatment for this diagnosis:**: {prompt_info['treatment']}

**Possible side effects of treatment**: {prompt_info['sideeffects']}"""


In [24]:
story=generate(text1,generation_config,safety_settings)

## La Aventura de Josué y el Tubito Mágico

Josué, un chico de quince años al que le encantaba bailar, montar a caballo, jugar al fútbol y ver anime japonés, se enfrentaba a una nueva aventura. No era una aventura como las de sus series favoritas, con viajes a otros mundos o batallas épicas, pero era una aventura importante para su salud. 

Los médicos le habían explicado a Josué y a sus padres que necesitaba un "catéter de Lehman". Era una palabra larga y complicada, pero Josué no se preocupó. Su imaginación ya estaba trabajando, convirtiendo esa palabra extraña en algo emocionante. 

"¿Un catéter?", se preguntó Josué. "Suena como el nombre de un personaje de anime. ¡Quizás sea como un robot que me ayudará a luchar contra los monstruos de las enfermedades!"

Su mamá le sonrió y le explicó: "No es exactamente un robot, Josué. Es más como un tubito mágico que nos ayudará a darte las medicinas que necesitas de una forma más fácil y rápida."

"¿Un tubito mágico?", Josué se imaginó un tubi

In [25]:
story

' ##  La Aventura de Josué y el Tubito Mágico\n\nJosué,  un chico de quince años al que le encantaba bailar, montar a caballo,  jugar al fútbol y ver anime japonés, se enfrentaba a una nueva aventura. No era una aventura como las de sus series favoritas, con viajes a otros mundos  o batallas épicas, pero era una aventura importante para su salud. \n\nLos médicos le habían explicado a Josué y a sus padres que necesitaba un  "catéter de Lehman". Era una palabra larga y complicada, pero Josué no se preocupó. Su imaginación ya estaba trabajando, convirtiendo esa palabra extraña en algo emocionante. \n\n"¿Un catéter?", se preguntó Josué.  "Suena como el nombre de un personaje de anime. ¡Quizás sea como un robot que me ayudará a luchar contra los monstruos de las enfermedades!"\n\nSu mamá le sonrió y le explicó: "No es exactamente un robot,  Josué. Es más como un tubito mágico que nos ayudará a darte las medicinas que necesitas de una forma más fácil y rápida."\n\n"¿Un tubito mágico?", Josué

In [26]:
f=open('story_Joshua.txt','w')
f.write(story)
f.close()